# Real Time Systems

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from rts.Helpers import *
from rts.Processor import *
from rts.Task import *
from rts.TaskSet import *
from timeit import default_timer as timer

### Liu-Layland-Test

In [2]:
T = TaskSet(PTask(3, 1), PTask(5, 2), PTask(8, 3))

print("-" * 50)
print("Result")
print("-" * 50)
print(f"LL-Test for task set T = {T}")
print(f"=> u <= uRM: {T.ll_test()}")
print("-" * 50)

--------------------------------------------------
Result
--------------------------------------------------
LL-Test for task set T = [T(3,1), T(5,2), T(8,3)]
u	= 1.1083
uRM	= 0.7798
=> u <= uRM: False
--------------------------------------------------


### RMA Test

Iteration rule:
$$
t^{(l+1)} = t_{e,i} * \sum_{k=1}^{i-1}{\lceil\frac{t^{(l)}}{t_{p,k}}\rceil * t_{e,k}}
$$

In [3]:
T = TaskSet(PTask(100, 30), PTask(150, 40), PTask(190, 35))

print("-" * 50)
print("Result")
print("-" * 50)
print(f"RMA Test for task set T = {T}")

res = T.rma_test()

print(f"=> t_wcrt < t_pmin: {res}")
print("-" * 50)

--------------------------------------------------
Result
--------------------------------------------------
RMA Test for task set T = [T(100,30), T(150,40), T(190,35)]
Least prioritized task: T(190,35)
t0	= 35
t1	= 105.0
t2	= 135.0
t3	= 135.0
=> converges after 4 iterations
t_wcrt	= 135.0
=> t_wcrt < t_pmin: True
--------------------------------------------------


### SR Test

Test for "Specialization operation" by Han/Tyan

In [4]:
T = TaskSet(PTask(6, 1), PTask(8, 3), PTask(10, 2), PTask(24, 1), PTask(49, 1))

print("-" * 50)
print("Result")
print("-" * 50)
print(f"SR Test for task set T = {T}")

start = timer()
res = T.sr_test()
end = round((timer() - start) * 1000000, 10)

res = pd.DataFrame(res).T
print(res)

print("-" * 50) 
print("Stats")
print("-" * 50)
print(f"Executed in {end} μs")

# fig = px.line(x=range(1, len(execution_times) + 1), y=execution_times)
# fig.update_layout(xaxis_title="Execution", yaxis_title="Execution time (μs)")
# fig.show()

--------------------------------------------------
Result
--------------------------------------------------
SR Test for task set T = [T(6,1), T(8,3), T(10,2), T(24,1), T(49,1)]
       tbi                                p_mod         u  u < 1
T0     6.0          [6.0, 6.0, 6.0, 24.0, 48.0]    1.0625  False
T1     4.0          [4.0, 8.0, 8.0, 16.0, 32.0]   0.96875   True
T2     5.0         [5.0, 5.0, 10.0, 20.0, 40.0]     1.075  False
T3     6.0          [6.0, 6.0, 6.0, 24.0, 48.0]    1.0625  False
T4  3.0625  [3.0625, 6.125, 6.125, 12.25, 49.0]  1.244898  False
--------------------------------------------------
Stats
--------------------------------------------------
Executed in 574.2920038756 μs


## MP-Scheduling

### RMNF

In [5]:
T = TaskSet(PTask(2, 1), PTask(2.5, 0.1), PTask(3, 1), PTask(4, 1), PTask(4.5, 0.1), PTask(5, 1), PTask(6, 1), PTask(7, 1), PTask(8, 1), PTask(8.5, 0.1), PTask(9, 1))
cpu = Processor(8)

In [6]:
success = cpu.rmnf(T)

print('-' * 50)
print("Result")
print('-' * 50)
print(f"RMNF for task set T = {T}")
print(f"uRM for T: {T.urm}")
df = pd.DataFrame(cpu.get_partitioning()).T
print(df)
print(f"Success: {success}")
print('-' * 50)

--------------------------------------------------
Result
--------------------------------------------------
RMNF for task set T = [T(2,1), T(2.5,0.1), T(3,1), T(4,1), T(4.5,0.1), T(5,1), T(6,1), T(7,1), T(8,1), T(8.5,0.1), T(9,1)]
uRM for T: 0.7154519838395894
           u       urm                                         Tasks
C1      0.54  0.828427                          [T(2,1), T(2.5,0.1)]
C2  0.638889  0.743492  [T(3,1), T(4,1), T(4.5,0.1), T(5,1), T(6,1)]
C3  0.247876  0.756828          [T(7,1), T(8,1), T(8.5,0.1), T(9,1)]
C4         0         1                                            []
C5         0         1                                            []
C6         0         1                                            []
C7         0         1                                            []
C8         0         1                                            []
Success: True
--------------------------------------------------


### RMFF

In [7]:
success = cpu.rmff(T)

print('-' * 50)
print("Result")
print('-' * 50)
print(f"RMFF for task set T = {T}")
print(f"uRM for T: {T.urm}")
df = pd.DataFrame(cpu.get_partitioning()).T
print(df)
print(f"Success: {success}")
print('-' * 50)

--------------------------------------------------
Result
--------------------------------------------------
RMFF for task set T = [T(2,1), T(2.5,0.1), T(3,1), T(4,1), T(4.5,0.1), T(5,1), T(6,1), T(7,1), T(8,1), T(8.5,0.1), T(9,1)]
uRM for T: 0.7154519838395894
           u       urm                                              Tasks
C1  0.740654  0.743492  [T(2,1), T(2.5,0.1), T(4.5,0.1), T(6,1), T(8.5...
C2   0.72619  0.779763                           [T(3,1), T(4,1), T(7,1)]
C3  0.436111  0.779763                           [T(5,1), T(8,1), T(9,1)]
C4         0         1                                                 []
C5         0         1                                                 []
C6         0         1                                                 []
C7         0         1                                                 []
C8         0         1                                                 []
Success: True
--------------------------------------------------


### RMFFDU

In [8]:
success = cpu.rmffdu(T)

print('-' * 50)
print("Result")
print('-' * 50)
print(f"RMFFDU for task set T = {T}")
print(f"uRM for T: {T.urm}")
df = pd.DataFrame(cpu.get_partitioning()).T
print(df)
print(f"Success: {success}")
print('-' * 50)

--------------------------------------------------
Result
--------------------------------------------------
RMFFDU for task set T = [T(8.5,0.1), T(4.5,0.1), T(2.5,0.1), T(9,1), T(8,1), T(7,1), T(6,1), T(5,1), T(4,1), T(3,1), T(2,1)]
uRM for T: 0.7154519838395894
    u urm                                              Tasks
C1  0   1  [T(8.5,0.1), T(4.5,0.1), T(2.5,0.1), T(9,1), T...
C2  0   1                                   [T(5,1), T(4,1)]
C3  0   1                                           [T(3,1)]
C4  0   1                                           [T(2,1)]
C5  0   1                                                 []
C6  0   1                                                 []
C7  0   1                                                 []
C8  0   1                                                 []
Success: True
--------------------------------------------------


### RMST

In [9]:
success = cpu.rmst(T)

print('-' * 50)
print("Result")
print('-' * 50)
print(f"RMNF for task set T = {T}")
print(f"uRM for T: {T.urm}")
df = pd.DataFrame(cpu.get_partitioning()).T
print(df)
print(f"Success: {success}")
print('-' * 50)

--------------------------------------------------
Result
--------------------------------------------------
RMNF for task set T = [T(8,1), T(4,1), T(2,1), T(8.5,0.1), T(4.5,0.1), T(9,1), T(5,1), T(2.5,0.1), T(6,1), T(3,1), T(7,1)]
uRM for T: 0.7154519838395894
           u urm                                             Tasks
C1  0.886765   1              [T(8,1), T(4,1), T(2,1), T(8.5,0.1)]
C2      0.54   1  [T(4.5,0.1), T(9,1), T(5,1), T(2.5,0.1), T(6,1)]
C3   0.47619   1                                  [T(3,1), T(7,1)]
C4         0   1                                                []
C5         0   1                                                []
C6         0   1                                                []
C7         0   1                                                []
C8         0   1                                                []
Success: True
--------------------------------------------------
